In [1]:
newtype Cont r a = Cont { (>>-) :: (a -> r) -> r }

In [2]:
instance Functor (Cont r) where
    fmap f cx = Cont $ \pur -> cx >>- (pur . f)

instance Applicative (Cont r) where
    pure    a = Cont $ \pur -> pur a
    (<*>) f a = Cont $ \pur ->
        f >>- \f' ->
        a >>- \a' ->
        pur (f' a')

instance Monad (Cont r) where
    (>>=) cx f = Cont $ \pur ->
        cx  >>- \a ->
        f a >>- pur

callCC :: ((b -> Cont r a) -> Cont r b) -> Cont r b
callCC f = Cont $ \pur ->
    f (\x -> (exit (pur x))) >>- \a ->
    pur a
    where exit x = Cont $ \_ -> x

In [3]:
newtype ContT r m a = ContT { (>>>-) :: (a -> m r) -> m r}

In [4]:
instance Functor (ContT r m) where
    fmap f cx = ContT $ \pur ->
        cx >>>- \a -> pur (f a)

instance Applicative (ContT r m) where
    pure a = ContT $ \pur -> pur a
    (<*>) f a = ContT $ \pur ->
        f >>>- \f' ->
        a >>>- \a' ->
        pur (f' a')    

instance Monad (ContT r m) where
    (>>=) cx f = ContT $ \pur ->
        cx  >>>- \a ->
        f a >>>- pur

callCC :: ((b -> ContT r m a) -> ContT r m b) -> ContT r m b
callCC f = ContT $ \pur ->
    f (\x -> (exit (pur x))) >>>- \a ->
    pur a
    where exit x = ContT $ \_ -> x

In [5]:
callCC' :: ((b -> ContT r m a) -> ContT r m b) -> ContT r m b
callCC' f = ContT $ \k -> (f (\b -> ContT $ \k' -> k b)) >>>- k